In [1]:
from datetime import datetime, timedelta
from sqlalchemy import create_engine, MetaData, Table
import pandas as pd
today = datetime.now()
yesterday = today - timedelta(days=3)
yesterday = yesterday.strftime('%Y-%m-%d')
engine = create_engine('mysql+pymysql://root@localhost/footviz')
df_partidos_ayer = pd.read_sql(f"SELECT * FROM partido WHERE fecha = '{yesterday}'", engine)


In [2]:
df_partidos_ayer

,id_partido,jornada,url_fbref,url_sofascore,fecha,temporada,equipo_local,equipo_visitante,estadio,arbitro
0,20,2,None,https://www.sofascore.com/football/match/liver...,2025-08-25,1,15,12,15,None
1,399,2,None,https://www.sofascore.com/football/match/athle...,2025-08-25,2,21,31,21,None
2,400,2,None,https://www.sofascore.com/football/match/getaf...,2025-08-25,2,38,28,38,None
3,769,1,None,https://www.sofascore.com/football/match/hella...,2025-08-25,3,58,49,58,None
4,770,1,None,https://www.sofascore.com/football/match/inter...,2025-08-25,3,50,57,41,None


In [1]:
from scrape_sofa import get_sofascore_api_data, init_driver
driver = init_driver()

data = get_sofascore_api_data(driver, path=f"api/v1/unique-tournament/8/season/77559/standings/total")


In [9]:
data['standings'][0]['rows'][0]['position']

1